In [ ]:
#역전파알고리즘: 출력층에서 입력층 방향으로 오차를 전파시키며 각 층의 가중치를 업데이트함(w값을 업데이트)
하나의 학습 데이터에 대한 비용함수의 그레디언트를 계산하고 미분을 효율적으로 계산할 수 있고 결과값의 오차도 적음
변수는 가중치이고 변수를 함수에 넣으면 아웃풋에서 오차가 나오는데 출력층->입력층 방향으로 오차를 전파시킨다
수치미분의 계산비용과 정확도를(수치미분은 근삿값으로 하기 때문에 정확도가 미흡.) 개선하였다.

#연쇄법칙
여러 함수를 사슬처럼 연결하여 사용. 연쇄법칙에 따라서 합성함수의 미분은 구성함수 각각을 미분하 후에 곱한 것과 같다.
입력값을 x라 할때 a는 w1, b는 w2라하고 y가 손실함수값이면
y는 (dy/dy) x는(dy/dx) a는 w1(dy/da), b는 w2(dy/db). A-> A'(x) B-> B'(a), C-> C'(b) 에 대응한다.

#역전파의 원리
합성함수의 미분은 구성 함수들의 미분의 곱으로 분해할 수 있다.
#손실함수
손실함수는 기계 학습 모델의 성능을 측정하기 위해 사용되는 함수이다.
모델의 예측값과 실졔 관측된 값 사이의 차이를 나타내며 차이가 작을수록 성능이 좋다(0이 목표)
손실함수의 각 매개 변수에 대한 미분계산.

#순전파 계산 그래프와 역전파 계산 그래프
순전파 계산그래프는 통상적인 계산이고 변수는 통상값이 존재한다
역전파 계산그래프는 마분값을 구하기 위한 계산이며 변수는 미분값이 존재한다.
역전파를 위해서는 순전파의 데이터가 필요하고 먼저 순전파를 한 뒤 역전파를 진행한다.(이때 각 함수가 입력 변수의 값을 기억해야함)

In [34]:
import numpy as np

In [35]:
#수동역전파 코드구현
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None #인스턴스 변수 추가.

In [36]:
#미분을 계산하는 역전파와 forward 메서드 호출 시 건네받은 Variable 인스턴스 유지
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input #입력 변수를 기억한다
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy): #역전파를 담당하는 백월드 메서드
        raise NotImpletedError()

In [37]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy #출력 쪽에서 전해지는 미분 값을 전달
        return gx

In [38]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy #역전파를 담당하는 메서드
        return gx

In [39]:
#순전파 계산그래프 (s->e->s)
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)


In [40]:
#역전파 계산 그래프
y.grad = np.array(1.0)
b.grad = C.backward(y.grad) #backward는 미분. a->b->c가 순전파이므로 역전파는 c->b->a로 진행
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [ ]:
#역전파를 자동화
순전파를 한번만 수행하면 역전파가 자동으로 이루어지는 구조.
(수동으로 조합하면 새로운 계산을 할 때마다 직접 작성해야하는 어려움이 있음)

#define-by-run :동적계산 그래프 
딥러닝에서 수행하는 계산들을 계산 시점에 체인처럼 연결하는 방식. 순전파를 한번 실행하면 역전파는 자동으로 수행.
분기가 있는 계산 그래프와 변수가 여러번 사용되는 복잡한 계산 그래프라도 역전파를 자동으로 할 수 있는 구조를 만들어야함.
#Q: <-> define-and-run(정적그래프)에 대하여 알아보기

#역전파 자동화
변수와 함수의 관계를 이해하는 데서 출발한다.
함수의 입장에서는 변수는 하나는 입력이고 출력이며 변수에게 있어서 함수는 창조자이다.
1)Variable 클래스에 creator 인스턴스 변수를 추가하고 set_creator메서드도 추가. 
output이라는 Variable 인스턴스에 output_creator(self)메서드를 호출하여 동적으로 연결.
연결된 Variable과 Function으로 계산 그래프를 거꾸로 거슬러 올라갈 수 있다.
    
#계산 그래프를 거꾸로 거슬러 올라가는 코드
assert문으로 조건을 충족하는지 확인하고 계산은 함수와 변수 사이의 연결로 구성. 실제로 계산을 수행하는 시점에 만들어짐.

#링크드 리스트 데이터 구조
노드들의 연결로 이루어진 데이터 구조이며 노드는 그래프를 구성하는 요소, 링크는 다른 노드를 가르키는 참조.
링크드 리스트 데이터 구조를 이용해 계싼 그래프 표현가능.

#역전파 자동화의 시작
1.함수를 가져옴 2.함수의 입력을 가져옴 3.함수의 backward 메서드 호출

In [62]:
import numpy as np

In [63]:
#역전파 자동화의 시작
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator  # 1.함수를 얻어옴
        if f is not None:
            x = f.input  # 2.입력변수를 가져옴
            x.grad = f.backward(self.grad)  # 3.자신보다 하나 앞에 놓인 변수의 backward메서드 호출
            x.backward()

In [64]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)  
        self.input = input
        self.output = output 
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [65]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [66]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


In [67]:
#순전파 계산그래프 (s->e->s)
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

assert y.creator == C 
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [68]:
y.grad = np.array(1.0)

C = y.creator
b = C.input
b.grad = C.backward(y.grad)

B = b.creator
a = B.input
a.grad = B.backward(b.grad)

A = a.creator
x = A.input
x.grad = A.backward(a.grad)

print(x.grad)

3.297442541400256


In [ ]:
#반복작업 자동화를 위한 backward 추가
Variable 클래스에 backward라는 새로운 메서드를 추가하여 반복작업을 자동화
#backward 메서드의 재귀절 호출
1)Variable의 creator에서 함수를 얻어옴
2)함수의 입력변수를 가져옴
3)함수 backward 메서드 호출
4)자신보다 하나 앞에 놓인 변수의 backward메서드 호출

In [69]:
import numpy as np

In [70]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator  # 1.함수를 얻어옴
        if f is not None:
            x = f.input  # 2.입력변수를 가져옴
            x.grad = f.backward(self.grad)  # 3.자신보다 하나 앞에 놓인 변수의 backward메서드 호출
            x.backward()

In [71]:
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [ ]:
#처리효율과 메서드 확장을 위해 backward 메서드 구현 방식을 재귀문에서 반복문으로 변경
재귀는 함수를 재귀적으로 호출할 때마다 중간 결괄ㄹ 메모리에 유지하면서 처리하므로 스택에 쌓여 효율이 좋지 않다.

#반복문을 이용한 구현 순서
1)함수들을 func 리스트에 차례로 집어넣음
2)while문 안에서 funcs.pop()을 호출하여 처리할 함수를 거냄
3)함수의 backward 메서드 호출
4)함수의 입력과 출력 변수를 얻음
5)얻은 입력 변수로 하나 앞의 함수를 리스트에 추가
리스트에 func를 집어넣고 pop으로 꺼내고 다시 입력

In [74]:
import numpy as np

In [85]:
#리스트를 이용하여 재귀문을 반복문으로 바꾼 코드
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self): #이부분이 다른부분.
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()  
            x, y = f.input, f.output 
            x.grad = f.backward(y.grad) 

            if x.creator is not None:
                funcs.append(x.creator)

In [86]:
#여기서부터는 이하동일 
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)  
        self.input = input
        self.output = output 
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [87]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [88]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [89]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# backward
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [ ]:
#세가지 개선사항
1)파이썬 함수로 이용하기
2)backward 메서드 간소화
3)ndarray만 취급하기

In [ ]:
# 1)파이썬 함수로 이용하기. 클래스 사용시 클래스의 인스턴스를 생성 후 그 인스턴스를 호출하는 두단계로 진행
x = Variable(np.array(0.5))
y = Square()(x)

In [ ]:
#파이썬 함수를 지원하는 방법: square 클리스를 square()함수로 구현, exp 클래스도 exp()함수로 구현
def square(x):
    return Square()(x)
def exp(x):
    return Exp()(x)

In [ ]:
#구현한 두 함수 사용시 코드변화
x = Variable(np.array(0.5))
y = square(exp(square(x)))

y.grad = np.array(1.0)
y.backward()
print(x.grad)

In [ ]:
# 2)backward 메서드 간소화 ++보완 필요.
'''
y.grad = np.array(1.0) 부분 생략하고 np.ones_like(self.data)코드로 변경
    self.data형상과 데이터 타입이 같은 ndarray 인스턴스 생성
    self.data가 스칼라라면 self.grad도 스칼라가됨
    Variable의 data와 grad의 데이터 타입을 같게 만들기 위함.


교수님 이부분 어려운데 너무 빨리설명하셔서 뭐라는지 모르겠어요...ㅠㅠㅠㅠㅠㅠㅠㅠㅠ

'''



In [ ]:
#테스트 프로그램의 중요성
버그를 예방할 수 있고
테스트를 자동화 해야 소프트웨어 품질을 유지한다.

#단위테스트
테스트 가능한 가장 작은 소프트웨어를 실행하여 예상대로 동작하는지 확인하는 테스트
테스트 대상 단위의 크기를 작게 설정해서 단위 테스트를 최대한 간단하게 작성
화이트박스 테스트
단위테스트는 TDD와 함께할 때 더 강력하다.
Java의 Junit(제대로 동작하는지 테스트프로그램을 통하여 비교)

#통합 테스트
여러 모듈을 모아 개발된 의도대로 동작되는지 확인하는 테스트
단위 테스트에서 발견하기 어려운 버그를 찾을 수 있는 장점
신뢰성이 떨어질 수 있는 점과 어디서 에러가 발생했는지 확인하기 쉽지 않음.